In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,TensorBoard

In [ ]:
file='/content/drive/MyDrive/data science projects/ct scan/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000000 (6).png'
sample_image=plt.imread(file)
plt.imshow(sample_image)

In [ ]:
sample_image.shape

In [ ]:

#data augmentation
train_file_directory='/content/drive/MyDrive/data science projects/ct scan/Data/train/'
valid_file_directory='/content/drive/MyDrive/data science projects/ct scan/Data/valid/'
test_file_directory='/content/drive/MyDrive/data science projects/ct scan/Data/test/'

train_data=ImageDataGenerator(rescale=(1./255),rotation_range=0.3,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.3,zoom_range=0.3)
test_data=ImageDataGenerator(rescale=(1./255),rotation_range=0.3,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.3,zoom_range=0.3)
valid_data=ImageDataGenerator(rescale=(1./255),rotation_range=0.3,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.3,zoom_range=0.3)


train_set=train_data.flow_from_directory(train_file_directory, target_size=(256,256), class_mode='categorical',batch_size=32)
test_set=test_data.flow_from_directory(test_file_directory, target_size=(256,256), class_mode='categorical',batch_size=32)
valid_set=valid_data.flow_from_directory(valid_file_directory, target_size=(256,256), class_mode='categorical',batch_size=32)

# Base model

In [ ]:
model_1 = Sequential()
model_1.add(Conv2D(input_shape=(256,256,3),filters=64,kernel_size=(3,3), activation="relu"))
model_1.add(Conv2D(filters=64,kernel_size=(3,3), activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2)))
model_1.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
model_1.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2)))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2)))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D(pool_size=(2,2)))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model_1.add(MaxPool2D())

model_1.add(Flatten())
model_1.add(Dense(units=4, activation="softmax"))


model_1.compile(loss='CategoricalCrossentropy', optimizer='adam',metrics=['accuracy'])
 
model_1.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=15,
                          restore_best_weights=True
                          )

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               patience=3,
                               min_lr=0.000001)

logdir='log/.'
tensorboard = TensorBoard(logdir, histogram_freq=1)
callbacks = [early_stopping,lr_reducer,tensorboard]

In [ ]:

model_1.fit_generator(
  train_set,
  epochs=50,
  steps_per_epoch=len(train_set),
  validation_data=valid_set,
  validation_steps=2,
  callbacks=callbacks)

In [ ]:
model_1.evaluate(test_set)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log

# feature extractor + predcitions -VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

In [ ]:
vgg=VGG16(input_shape=(256,256,3),include_top=False)
vgg.summary()

In [ ]:
vgg.trainable=True

In [ ]:
for layer in vgg.layers:
  print(layer, layer.trainable)

In [ ]:
x = Flatten()(vgg.output)
x=BatchNormalization()(x)
prediction = Dense(4, activation='softmax')(x)

model_2 = Model(inputs=vgg.input, outputs=prediction)
model_2.summary()

In [ ]:
model_2.compile(loss='CategoricalCrossentropy', optimizer='adam',metrics=['accuracy'])


model_2.fit_generator(
  train_set,
  epochs=100,
  steps_per_epoch=len(train_set),
  validation_data=test_set,
  validation_steps=10,
  callbacks=callbacks)

In [ ]:
model_2.evaluate(valid_set)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log

# fine tuning


In [ ]:
vgg=VGG16(input_shape=(256,256,3),include_top=False)
vgg.summary()

In [ ]:
for layer in vgg.layers[:-2]:
  layer.trainable=False

for layer in vgg.layers:
  print(layer,layer.trainable)

In [ ]:
y = Flatten()(vgg.output)
y=BatchNormalization()(y)
prediction = Dense(4, activation='softmax')(y)
model_3 = Model(inputs=vgg.input, outputs=prediction)
model_3.summary()

In [ ]:
model_3.compile(loss='CategoricalCrossentropy', optimizer='adam',metrics=['accuracy'])


model_3.fit_generator(
  train_set,
  epochs=100,
  steps_per_epoch=len(train_set),
  validation_data=test_set,
  validation_steps=10,
  callbacks=callbacks)

In [ ]:
%tensorboard --logdir log

# vanilla vgg16

In [ ]:
vgg=VGG16(input_shape=(256,256,3),include_top=False)
vgg.trainable=False

for layer in vgg.layers:
  print(layer,layer.trainable)

In [ ]:
z = Flatten()(vgg.output)
z=BatchNormalization()(z)
prediction = Dense(4, activation='softmax')(z)

model_4 = Model(inputs=vgg.input, outputs=prediction)
model_4.summary()

In [ ]:
model_4.compile(loss='CategoricalCrossentropy', optimizer='adam',metrics=['accuracy'])


model_4.fit_generator(
  train_set,
  epochs=100,
  steps_per_epoch=len(train_set),
  validation_data=test_set,
  validation_steps=10,
  callbacks=callbacks)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log


In [ ]:
print("Model 1 ",model_1.evaluate(valid_set))
print("Model 2 ",model_2.evaluate(valid_set))
print("Model 3 ",model_3.evaluate(valid_set))
print("Model 4 ",model_4.evaluate(valid_set))

In [ ]:
model_3.save('/content/drive/MyDrive/data science projects/ct scan/model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image


test_file='/content/drive/MyDrive/data science projects/ct scan/Data/valid/normal/003828_02_01_174.png'
test_image=image.load_img(test_file,target_size=(256,256))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

In [ ]:
result=np.argmax(model_3.predict(test_image))
result


In [ ]:
targets={
    0 : 'adenocarcinimo', 
    1: 'large cell',
    2 : 'normal',
    3: 'squamous cell'
}

In [ ]:
for key, values in targets.items():
  if key==result:
    print(values)